In [62]:
#import packages and set options
import string
import feedparser
import pandas as pd
#will display maximum rows.
from IPython.core.display import display, HTML
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

#function to display all columns and rows from: https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook
def force_show_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(HTML(df.to_html()))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\William\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
#Get RSS Feeds and put them into a list of feedparsers
rawrss = [
    'http://rss.cnn.com/rss/cnn_topstories.rss',    #cnn
    'https://rss.nytimes.com/services/xml/rss/nyt/US.xml', #nytimes
    'http://feeds.foxnews.com/foxnews/latest', #foxnews
    'http://feeds.washingtonpost.com/rss/politics?tid=lk_inline_manual_2&itid=lk_inline_manual_2', #washington post
    'https://feeds.npr.org/1001/rss.xml', #npr
    'http://feeds.reuters.com/reuters/topNews', #reuters
    'http://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml?edition=int' #BBC america
    ]

feeds = [] # list of feed objects
for url in rawrss:
    feeds.append(feedparser.parse(url))

In [64]:
import datetime
from dateutil import parser
#from datetime import strptime
posts = [] # list of posts [(title1, link1, summary1), (title2, link2, summary2) ... ]

for feed in feeds:
    try: 
        source = feed.feed.feedburner_info['uri']
    except AttributeError:
        source = feed.feed.link
    for post in feed.entries:
        try:
            date = post.published
          #  print(date)
          #  parsedDate = post.published.split(' ')
          #  date = datetime.datetime(parsedDate[3], parsedDate[2], parsedDate[1], parsedDate[4])
          #  print(date)
            #date = dateTime            
        except AttributeError:
            date = "no date"
        try:
            summary = post.summary
        except AttributeError:
            summary = "NA"
        posts.append((post.title, post.link, summary, date, source))


df = pd.DataFrame(posts, columns=['title', 'link', 'summary', 'date', 'source']) # pass data to df

def fixDate(date):
      try:
        date = parser.parse(date)
        return date
      except:
        return date  
    
df['date'] = df['date'].apply(lambda x: fixDate(x))



C:\Users\William\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [65]:
#begin cleaning data for analysis
#get rid of junk data in the summary
df['summary'] = df['summary'].apply(lambda x : (x.split('<',1)[0]))

#create cleaned column for the comboniation of the summary column and 
df['cleaned'] = df['title']+' '+df['summary']
print(df['cleaned'][0])

#lower df['cleaned']
df['cleaned']= df['cleaned'].str.lower()

#get red of uneeded characters
df['cleaned'] = df['cleaned'].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", x))



After admitting US deaths could hit 90,000, Trump bemoaned his own plight at the Lincoln Memorial Podcast | Timeline | Catch up | All 50 states' reopening plans


In [66]:
#provide sentiment anaylsis!
sia = SIA()
df['sentiment#'] = df['cleaned'].apply(lambda x: sia.polarity_scores(x))
df['sentiment#'] = df['sentiment#'].apply(lambda x: x['compound'])
#df['sentiment#']

In [67]:
def sentiment(compound):
    if compound > .2:
        return 'positive'
    elif compound < -.2:
        return 'negative'
    else:
        return 'neutral'
df['sentiment'] = df['sentiment#'].apply( lambda x: sentiment(x))
#force_show_all(df)
#df.cleaned
                                        

In [68]:

#Combine with old articles
print(len(df))
old_df=pd.read_csv('Articles.csv')
print(len(old_df))
df['trump'] = 0;
df['biden'] = 0;
df = pd.concat([df,old_df]).drop_duplicates(subset='link', keep='first').reset_index(drop=True)
df = df.drop_duplicates(subset='title', keep='first')
print(len(df))

#df.to_csv('articlescheck.csv', index=False)

180
1245
1357


In [69]:
#for ind in df.index:
 #   if('trump' in df['cleaned'][ind]):
 #       df['trump'][ind]=1
 #   if('biden' in df['cleaned'][ind]):
 #       df['biden'][ind]=1

def checkTrump(text):
    if 'trump' in text:
        return 1
    else:
        return 0
def checkBiden(text):
    if 'biden' in text:
        return 1
    else:
        return 0
df['trump'] = df['cleaned'].apply(lambda x: checkTrump(x))
df['biden'] = df['cleaned'].apply(lambda x: checkBiden(x))

force_show_all(df)

df.to_csv('Articles.csv', index=False)


,title,link,summary,date,source,cleaned,sentiment#,sentiment,trump,biden
0,"After admitting US deaths could hit 90,000, Trump bemoaned his own plight at the Lincoln Memorial",http://rss.cnn.com/~r/rss/cnn_topstories/~3/o0g0MrdjZpI/index.html,Podcast | Timeline | Catch up | All 50 states' reopening plans,2020-05-04 11:29:16+00:00,rss/cnn_topstories,after admitting us deaths could hit 90000 trump bemoaned his own plight at the lincoln memorial podcast timeline catch up all 50 states reopening plans,0.0000,neutral,1,0
1,States reopen as pandemic persists,http://rss.cnn.com/~r/rss/cnn_topstories/~3/p9RFNw81HB4/index.html,,2020-05-04 13:05:12+00:00,rss/cnn_topstories,states reopen as pandemic persists,0.0000,neutral,0,0
2,Japan extends state of emergency,http://rss.cnn.com/~r/rss/cnn_topstories/~3/WTwrKcy7m_o/index.html,,no date,rss/cnn_topstories,japan extends state of emergency,-0.2732,negative,0,0
3,US stocks open the week lower,http://rss.cnn.com/~r/rss/cnn_topstories/~3/Xl04zMse4-w/index.html,,2020-05-04 13:41:55+00:00,rss/cnn_topstories,us stocks open the week lower,-0.2960,negative,0,0
4,Fact check: Trump peppers Fox News town hall with false claims,http://rss.cnn.com/~r/rss/cnn_topstories/~3/XmhtY3E_hBo/index.html,"President Donald Trump delivered another rapid series of false and misleading claims on Sunday night, this time at a Fox News ""virtual town hall"" event at the Lincoln Memorial in Washington, DC.",2020-05-04 10:17:42+00:00,rss/cnn_topstories,fact check trump peppers fox news town hall with false claims president donald trump delivered another rapid series of false and misleading claims on sunday night this time at a fox news virtual town hall event at the lincoln memorial in washington dc,-0.4019,negative,1,0
5,See where your state stands on flattening the curve,http://rss.cnn.com/~r/rss/cnn_topstories/~3/7j-wYoq2Lkw/index.html,,no date,rss/cnn_topstories,see where your state stands on flattening the curve,0.0000,neutral,0,0
6,King breaks down which states meet guidelines for reopening,http://rss.cnn.com/~r/rss/cnn_topstories/~3/-IXMc2JVtkE/states-reopening-john-king-magic-wall-ip-vpx.cnn,CNN's John King takes a look at how all 50 states are addressing reopening amid the coronavirus pandemic.,2020-05-03 16:35:39+00:00,rss/cnn_topstories,king breaks down which states meet guidelines for reopening cnns john king takes a look at how all 50 states are addressing reopening amid the coronavirus pandemic,0.0000,neutral,0,0
7,Brian Stelter: Leaders are failing to tell the hard truth,http://rss.cnn.com/~r/rss/cnn_topstories/~3/6ZTY6VhC0Rs/stelter-covid-19-coronavirus-opening-hard-truths-rs-vpx.cnn,CNN's Brian Stelter breaks down the hard truths some leaders are refusing to talk about with the public.,2020-05-03 15:41:54+00:00,rss/cnn_topstories,brian stelter leaders are failing to tell the hard truth cnns brian stelter breaks down the hard truths some leaders are refusing to talk about with the public,-0.4019,negative,0,0
8,'More deaths than we could have ever imagined': Largest US city deals with its dead,http://rss.cnn.com/~r/rss/cnn_topstories/~3/OJ9GnV2XPWU/index.html,,2020-05-04 00:40:40+00:00,rss/cnn_topstories,more deaths than we could have ever imagined largest us city deals with its dead,-0.6486,negative,0,0
9,Supreme Court offers oral arguments live for the first time,http://rss.cnn.com/~r/rss/cnn_topstories/~3/lQa4tkOJKSE/live-video-1,,no date,rss/cnn_topstories,supreme court offers oral arguments live for the first time,0.2263,positive,0,0


PermissionError: [Errno 13] Permission denied: 'Articles.csv'

In [70]:
#tokenize cleaned column for freqdist
# get stop words from NLTK and add my own stopwords
from collections import Counter
stop = stopwords.words('english')
sw_list= ["'", ',','said', 'people', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'could', 'says', ',',
          'e', 'n', 'r', 'l', 'h', 'c', 'u','p','g','f','w','v','b',"'",'poop', "'", ',', 'new']
stop.extend(sw_list)
punct = nltk.tokenize.word_tokenize(string.punctuation)
stop.extend(punct)
stopwords_dict = Counter(stop)
#print(stopwords_dict)
text = df['cleaned'].sum()
text = ' '.join([word for word in text.split() if word not in stopwords_dict])
text.replace
tokens = nltk.tokenize.word_tokenize(text)
#create frequency distribution 
word_dist = nltk.FreqDist(tokens)
#display top 20
word_dist.most_common(30)

[('coronavirus', 563),
 ('us', 287),
 ("'", 268),
 ('trump', 202),
 ('pandemic', 171),
 ('president', 161),
 (',', 153),
 ('covid19', 142),
 ('states', 127),
 ('first', 98),
 ('health', 91),
 ('biden', 90),
 ('house', 89),
 ('one', 83),
 ('state', 80),
 ('york', 79),
 ('may', 73),
 ('virus', 67),
 ('joe', 66),
 ('former', 66),
 ('reopen', 61),
 ('time', 60),
 ('news', 59),
 ('donald', 58),
 ('world', 58),
 ('say', 58),
 ('lockdown', 57),
 ('would', 56),
 ('public', 54),
 ('home', 54)]

In [71]:
import re
import os
import tweepy
import json
import csv
from textblob import TextBlob
import string
import preprocessor as p

In [72]:
#Twitter credentials for the app
consumer_key = 'LkrwAFTw0J1HvqfnWJ6o5ILQr'
consumer_secret = 'jj5Ok4lCnPRFeD5qXUy1c8NUcuFYBTZAlEz9ahwgebSh9QnyKa'
access_key= '598706178-AJRic3UjvSxl4BLWx2cvHULwQHsGEGyT8UhDbZhH'
access_secret = 'rdxLY0WGDK0XU8MOBUNF7X0Pyb53FMc4lHIU0YUBbv0fB'

#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [73]:
#declare file paths as follows for three files
trump_tweets = "data/trump_data.csv"
biden_tweets = "data/biden_data.csv"
#source: https://towardsdatascience.com/extracting-twitter-data-pre-processing-and-sentiment-analysis-using-python-3-0-7192bd8b47cf

#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
'favorite_count', 'retweet_count', 'original_author',   'possibly_sensitive', 'hashtags',
'user_mentions', 'place', 'place_coord_boundaries']

#set two date variables for date range
start_date = '2020-01-01'
end_date = '2020-04-29'

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3' ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [74]:
def clean_tweets(tweet): 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet


In [75]:
#method write_tweets()
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date).pages(50):
        for status in page:
            new_entry = []
            status = status._json

            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'en':
                continue

            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue


           #tweepy preprocessing called for basic preprocessing
            clean_text = (status['text'])

            #call clean_tweet method for extra preprocessing
            filtered_tweet=clean_tweets(clean_text)

            #pass textBlob method for sentiment calculations
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment

            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity

            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]

            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])

            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)

            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)

            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)

            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)

            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")


In [76]:
#declare keywords as a query for three categories
Trump_keywords = '#trump OR #donald OR #president OR #whitehouse'
Biden_keywords = '#biden OR #joebiden OR #democraft'

In [77]:
#call main method passing keywords and file path
write_tweets(Trump_keywords,  trump_tweets)
write_tweets(Biden_keywords, biden_tweets)

In [32]:
from platform import python_version
print(python_version())

3.7.3


'python3.7.3' is not recognized as an internal or external command,
operable program or batch file.


<class 'list'>


TypeError: 'WordListCorpusReader' object is not iterable